In [36]:
!git log -1

commit 2213ea5854e8855191c347c0409500aba66d3173
Author: John McCallum <cfljam@users.noreply.github.com>
Date:   Tue Feb 23 14:52:01 2016 +1300

    updated notebook


In [37]:
!git remote -v

origin	https://github.com/cfljam/pcr_marker_design (fetch)
origin	https://github.com/cfljam/pcr_marker_design (push)
upstream	https://github.com/PlantandFoodResearch/pcr_marker_design (fetch)
upstream	https://github.com/PlantandFoodResearch/pcr_marker_design (push)


In [24]:
!pip freeze

anaconda-client==1.6.1
appnope==0.1.0
cffi==1.9.1
clyent==1.2.2
conda==4.2.13
cryptography==1.7.1
decorator==4.0.11
entrypoints==0.2.2
idna==2.2
ipykernel==4.5.2
ipython==5.2.2
ipython-genutils==0.1.0
ipywidgets==5.2.2
Jinja2==2.8
jsonschema==2.5.1
jupyter-client==5.0.0
jupyter-console==5.1.0
jupyter-core==4.3.0
MarkupSafe==0.23
mistune==0.7.3
nbconvert==4.2.0
nbformat==4.2.0
notebook==4.4.1
numpy==1.12.0
pandas==0.19.2
pexpect==4.2.1
pickleshare==0.7.3
prompt-toolkit==1.0.13
ptyprocess==0.5.1
pyasn1==0.1.9
pycosat==0.6.1
pycparser==2.17
pycrypto==2.6.1
Pygments==2.2.0
pyOpenSSL==16.2.0
python-dateutil==2.3
pytz==2016.10
PyYAML==3.12
pyzmq==16.0.2
qtconsole==4.2.1
requests==2.12.4
ruamel-yaml===-VERSION
simplegeneric==0.8.1
six==1.10.0
terminado==0.6
tornado==4.4.2
traitlets==4.3.1
wcwidth==0.1.7
widgetsnbextension==1.2.6
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
pwd

'/Users/johnmccallum/Documents/pcr_marker_design/docs'

### Load up bedtools 

In [1]:

from pybedtools import BedTool
from pyfaidx import Fasta

## Base this on the galaxy  test data

Note that this is wrong!

1. indel lengths arent correct
2. should be 1-based and start==stop for SNPs

need to revisit vcf2gvf

In [2]:
!head  -n 4 ../test/test-data/targets.gff

k69_93535	SAMTOOLS	INDEL	3	7	4.44	.	.	ID=k69_93535:SAMTOOLS:INDEL:3;Variant_seq=ATTTCACATCCCATCATCA,ATCACATCCCATCATCA;Reference_seq=ATCA;INDEL;DP=5;VDB=0.0149;AF1=1;AC1=24;DP4=0,0,3,0;MQ=23;FQ=-33.2
k69_93535	SAMTOOLS	INDEL	5	6	4.91	.	.	ID=k69_93535:SAMTOOLS:INDEL:5;Variant_seq=CTCATATCCCATCATC,CCGGCC;Reference_seq=C;INDEL;DP=6;VDB=0.0137;AF1=1;AC1=24;DP4=0,0,3,1;MQ=17;FQ=-32.4
k69_93535	SAMTOOLS	SNP	1141	1142	999	.	.	ID=k69_93535:SAMTOOLS:SNP:1141;Variant_seq=T;Reference_seq=C;DP=2644;VDB=0.0374;AF1=0.1882;AC1=5;DP4=748,786,225,294;MQ=42;FQ=999;PV4=0.037,0,0.036,0.39
k69_93535	SAMTOOLS	SNP	1147	1148	999	.	.	ID=k69_93535:SAMTOOLS:SNP:1147;Variant_seq=G;Reference_seq=C;DP=2645;VDB=0.0371;AF1=0.3527;G3=0.2771,0.7229,6.934e-153;HWE=0.0248;AC1=8;DP4=733,804,447,519;MQ=42;FQ=999;PV4=0.51,0,0.027,1


### Create a Bed Target Set for SNPs only

In [3]:
!awk -v OFS='\t' '/SNP/ {print $1,$4-1,$4}' ../test/test-data/targets.gff | tee  ../test/test-data/targets.snps.bed

k69_93535	1140	1141
k69_93535	1146	1147
k69_93535	1335	1336
k69_93535	2	3
k69_93535	660	661
k69_93535	750	751
k69_93535	886	887
k69_98089	1	2
k69_98089	29	30
k69_98089	2	3
k69_98089	452	453
k69_98089	514	515
k69_98089	521	522
k69_98089	549	550
k69_98089	624	625
k69_98089	692	693
k69_98089	716	717
k69_98089	722	723
k69_98089	725	726


Use the default Globals settings for Primer3

In [4]:
p3_globals={
        'PRIMER_OPT_SIZE': 20,
        'PRIMER_PICK_INTERNAL_OLIGO': 0,
        'PRIMER_INTERNAL_MAX_SELF_END': 8,
        'PRIMER_MIN_SIZE': 18,
        'PRIMER_MAX_SIZE': 25,
        'PRIMER_OPT_TM': 60.0,
        'PRIMER_MIN_TM': 57.0,
        'PRIMER_MAX_TM': 63.0,
        'PRIMER_MIN_GC': 20.0,
        'PRIMER_MAX_GC': 80.0,
        'PRIMER_MAX_POLY_X': 100,
        'PRIMER_INTERNAL_MAX_POLY_X': 100,
        'PRIMER_SALT_MONOVALENT': 50.0,
        'PRIMER_DNA_CONC': 50.0,
        'PRIMER_MAX_NS_ACCEPTED': 0,
        'PRIMER_MAX_SELF_ANY': 12,
        'PRIMER_MAX_SELF_END': 8,
        'PRIMER_PAIR_MAX_COMPL_ANY': 12,
        'PRIMER_PAIR_MAX_COMPL_END': 8,
        'PRIMER_PRODUCT_SIZE_RANGE': [60,250],
    }

### Load up the Module and Create a Designer Object

In [5]:
from pcr_marker_design import design as d
test_seq='../test/test-data/targets.fasta'
annfile='../test/test-data/targets.snps.bed'
des_test=d.PrimerDesign(test_seq,annfile,'Designer Test')

### Make a BedTool target

In [6]:
testBed=BedTool('../test/test-data/targets.snps.bed')

In [7]:
[[X.length,X.start,X.end] for X in testBed]

[[1, 1140, 1141],
 [1, 1146, 1147],
 [1, 1335, 1336],
 [1, 2, 3],
 [1, 660, 661],
 [1, 750, 751],
 [1, 886, 887],
 [1, 1, 2],
 [1, 29, 30],
 [1, 2, 3],
 [1, 452, 453],
 [1, 514, 515],
 [1, 521, 522],
 [1, 549, 550],
 [1, 624, 625],
 [1, 692, 693],
 [1, 716, 717],
 [1, 722, 723],
 [1, 725, 726]]

### Create a Target

- base this on the test data targets file
- do this manually with the from_string option
- debtools is zero-indexed

In [8]:
target=BedTool("k69_93535 1146 1147",from_string=True)
print(target)

k69_93535	1146	1147



In [9]:
target[0].start

1146

In [10]:
target[0].end

1147

In [11]:
target[0].length

1

In [12]:
p3_globals['PRIMER_PRODUCT_SIZE_RANGE'][1]

250

In [13]:
seq_test=des_test.getseqslicedict(target,p3_globals['PRIMER_PRODUCT_SIZE_RANGE'][1])
seq_test

{'REF_OFFSET': 896,
 'SEQUENCE_EXCLUDED_REGION': [(244, 1), (439, 1)],
 'SEQUENCE_ID': 'Designer Test',
 'SEQUENCE_TARGET': (250, 1),
 'SEQUENCE_TEMPLATE': 'AAATAATGGAGAATAGATGGTTCAAGAATGGATTCGAGCCTGTGAAATATTACATTGAGAATGATAGGTTTCATAAGTGGTGTAGCTTAGACGAAGAGAATGCTAATGACAACGAGGAGGTAGAATCTGGAGATGAATCAGACTCTTCAGTTGCTTCCTGCCCTCCTACACTTAATGAAGGAAAGAAAAAAAGGACAGGGAAGCTTCATAGGCCTTTGAGTCTGAACGCATTTGACATAATTTCCTTTTCCAGAGGATTTGATCTTTCAGGTTTGTTTGAAGAAACGGGAGATGAAACAAGATTTGTGTCGGGTGAAACGATACCAAACATCATATCGAAATTGGAGGAGATTGCAAAAGTGGGTAGTTTCACGTTTAGGAAGAAGGATTGTAGGGTTAGTTTAGAAGGAACGCGAGAAGGAGTGAAGGGCCCTCTTACGATTGGAGCTGAGATATTTGAGCTTACGCCTAGTTTGGTTGTTGTTGAGCTTAAGAAGAAAG',
 'TARGET_ID': 'k69_93535:1147-1147'}

In [14]:
str(seq_test['TARGET_ID'])

'k69_93535:1147-1147'

In [15]:
import primer3 as P3
import pandas as pd

### Design Using the Slice and P3 py directly

In [16]:
P3_test=P3.designPrimers(seq_args=seq_test,
                 global_args= p3_globals)

In [17]:
test=pd.DataFrame.from_dict(P3_test)
test.PRIMER_LEFT_NUM_RETURNED

0    5
1    5
Name: PRIMER_LEFT_NUM_RETURNED, dtype: int64

Issues
====

- need to pass in more metadata
- id unless there is one in the bed tool
- target sequence and region so we can map back
- need a convenient tabular form to filter in pandas

### Design Using Module

In [18]:
from pcr_marker_design import run_p3

In [19]:
run_p3.run_P3(global_dict=p3_globals,target_dict=seq_test)

[{'PRIMER_LEFT': (1090, 20),
  'PRIMER_LEFT_SEQUENCE': 'ACAGGGAAGCTTCATAGGCC',
  'PRIMER_RIGHT': (1320, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CACTCCTTCTCGCGTTCCTT',
  'TARGET_ID': 'k69_93535:1147-1147'},
 {'PRIMER_LEFT': (1090, 20),
  'PRIMER_LEFT_SEQUENCE': 'ACAGGGAAGCTTCATAGGCC',
  'PRIMER_RIGHT': (1324, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CCTTCACTCCTTCTCGCGTT',
  'TARGET_ID': 'k69_93535:1147-1147'},
 {'PRIMER_LEFT': (1090, 20),
  'PRIMER_LEFT_SEQUENCE': 'ACAGGGAAGCTTCATAGGCC',
  'PRIMER_RIGHT': (1322, 20),
  'PRIMER_RIGHT_SEQUENCE': 'TTCACTCCTTCTCGCGTTCC',
  'TARGET_ID': 'k69_93535:1147-1147'},
 {'PRIMER_LEFT': (1105, 20),
  'PRIMER_LEFT_SEQUENCE': 'AGGCCTTTGAGTCTGAACGC',
  'PRIMER_RIGHT': (1320, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CACTCCTTCTCGCGTTCCTT',
  'TARGET_ID': 'k69_93535:1147-1147'}]

## Modified to return primer positions in Primer3 format with offset

In [20]:
pd.DataFrame.from_dict(run_p3.run_P3(global_dict=p3_globals,target_dict=seq_test))

,PRIMER_LEFT,PRIMER_LEFT_SEQUENCE,PRIMER_RIGHT,PRIMER_RIGHT_SEQUENCE,TARGET_ID
0,"(1090, 20)",ACAGGGAAGCTTCATAGGCC,"(1320, 20)",CACTCCTTCTCGCGTTCCTT,k69_93535:1147-1147
1,"(1090, 20)",ACAGGGAAGCTTCATAGGCC,"(1324, 20)",CCTTCACTCCTTCTCGCGTT,k69_93535:1147-1147
2,"(1090, 20)",ACAGGGAAGCTTCATAGGCC,"(1322, 20)",TTCACTCCTTCTCGCGTTCC,k69_93535:1147-1147
3,"(1105, 20)",AGGCCTTTGAGTCTGAACGC,"(1320, 20)",CACTCCTTCTCGCGTTCCTT,k69_93535:1147-1147


### Try Design around a longer Target INterval

use this magic to force reload when editing source

In [21]:
%load_ext autoreload
%autoreload 2

In [22]:
from pcr_marker_design import design as d
test_seq="../test/test-data/targets.fasta"
annfile= "../test/test-data/targets.snps.bed"
designer = d.PrimerDesign(test_seq,annfile,'Test')
my_target=BedTool("k69_93535 1200 1300",from_string=True)
seqslice=designer.getseqslicedict(target,250)

In [23]:
seqslice

{'REF_OFFSET': 896,
 'SEQUENCE_EXCLUDED_REGION': [(244, 1), (439, 1)],
 'SEQUENCE_ID': 'Test',
 'SEQUENCE_TARGET': (250, 1),
 'SEQUENCE_TEMPLATE': 'AAATAATGGAGAATAGATGGTTCAAGAATGGATTCGAGCCTGTGAAATATTACATTGAGAATGATAGGTTTCATAAGTGGTGTAGCTTAGACGAAGAGAATGCTAATGACAACGAGGAGGTAGAATCTGGAGATGAATCAGACTCTTCAGTTGCTTCCTGCCCTCCTACACTTAATGAAGGAAAGAAAAAAAGGACAGGGAAGCTTCATAGGCCTTTGAGTCTGAACGCATTTGACATAATTTCCTTTTCCAGAGGATTTGATCTTTCAGGTTTGTTTGAAGAAACGGGAGATGAAACAAGATTTGTGTCGGGTGAAACGATACCAAACATCATATCGAAATTGGAGGAGATTGCAAAAGTGGGTAGTTTCACGTTTAGGAAGAAGGATTGTAGGGTTAGTTTAGAAGGAACGCGAGAAGGAGTGAAGGGCCCTCTTACGATTGGAGCTGAGATATTTGAGCTTACGCCTAGTTTGGTTGTTGTTGAGCTTAAGAAGAAAG',
 'TARGET_ID': 'k69_93535:1147-1147'}

In [24]:
run_p3.run_P3(global_dict=p3_globals,target_dict=seqslice)

[{'PRIMER_LEFT': (1090, 20),
  'PRIMER_LEFT_SEQUENCE': 'ACAGGGAAGCTTCATAGGCC',
  'PRIMER_RIGHT': (1320, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CACTCCTTCTCGCGTTCCTT',
  'TARGET_ID': 'k69_93535:1147-1147'},
 {'PRIMER_LEFT': (1090, 20),
  'PRIMER_LEFT_SEQUENCE': 'ACAGGGAAGCTTCATAGGCC',
  'PRIMER_RIGHT': (1324, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CCTTCACTCCTTCTCGCGTT',
  'TARGET_ID': 'k69_93535:1147-1147'},
 {'PRIMER_LEFT': (1090, 20),
  'PRIMER_LEFT_SEQUENCE': 'ACAGGGAAGCTTCATAGGCC',
  'PRIMER_RIGHT': (1322, 20),
  'PRIMER_RIGHT_SEQUENCE': 'TTCACTCCTTCTCGCGTTCC',
  'TARGET_ID': 'k69_93535:1147-1147'},
 {'PRIMER_LEFT': (1105, 20),
  'PRIMER_LEFT_SEQUENCE': 'AGGCCTTTGAGTCTGAACGC',
  'PRIMER_RIGHT': (1320, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CACTCCTTCTCGCGTTCCTT',
  'TARGET_ID': 'k69_93535:1147-1147'}]

In [25]:
pd.DataFrame.from_dict(run_p3.run_P3(global_dict=p3_globals,target_dict=seqslice))

,PRIMER_LEFT,PRIMER_LEFT_SEQUENCE,PRIMER_RIGHT,PRIMER_RIGHT_SEQUENCE,TARGET_ID
0,"(1090, 20)",ACAGGGAAGCTTCATAGGCC,"(1320, 20)",CACTCCTTCTCGCGTTCCTT,k69_93535:1147-1147
1,"(1090, 20)",ACAGGGAAGCTTCATAGGCC,"(1324, 20)",CCTTCACTCCTTCTCGCGTT,k69_93535:1147-1147
2,"(1090, 20)",ACAGGGAAGCTTCATAGGCC,"(1322, 20)",TTCACTCCTTCTCGCGTTCC,k69_93535:1147-1147
3,"(1105, 20)",AGGCCTTTGAGTCTGAACGC,"(1320, 20)",CACTCCTTCTCGCGTTCCTT,k69_93535:1147-1147


### Fix up Test Data

In [26]:
P3.runP3Design

<function primer3.bindings.runP3Design>

In [27]:
p3_test_seq = {
        'TARGET_ID' : 'MH1000:',
        'SEQUENCE_ID': 'MH1000',
        'SEQUENCE_TEMPLATE': 'GCTTGCATGCCTGCAGGTCGACTCTAGAGGATCCCCCTACATTTT'
                             'AGCATCAGTGAGTACAGCATGCTTACTGGAAGAGAGGGTCATGCA'
                             'ACAGATTAGGAGGTAAGTTTGCAAAGGCAGGCTAAGGAGGAGACG'
                             'CACTGAATGCCATGGTAAGAACTCTGGACATAAAAATATTGGAAG'
                             'TTGTTGAGCAAGTAAAAAAAATGTTTGGAAGTGTTACTTTAGCAA'
                             'TGGCAAGAATGATAGTATGGAATAGATTGGCAGAATGAAGGCAAA'
                             'ATGATTAGACATATTGCATTAAGGTAAAAAATGATAACTGAAGAA'
                             'TTATGTGCCACACTTATTAATAAGAAAGAATATGTGAACCTTGCA'
                             'GATGTTTCCCTCTAGTAG',
        'SEQUENCE_INCLUDED_REGION': [36, 342]}

p3_test_globals = {
        'PRIMER_OPT_SIZE': 20,
        'PRIMER_PICK_INTERNAL_OLIGO': 1,
        'PRIMER_INTERNAL_MAX_SELF_END': 8,
        'PRIMER_MIN_SIZE': 18,
        'PRIMER_MAX_SIZE': 25,
        'PRIMER_OPT_TM': 60.0,
        'PRIMER_MIN_TM': 57.0,
        'PRIMER_MAX_TM': 63.0,
        'PRIMER_MIN_GC': 20.0,
        'PRIMER_MAX_GC': 80.0,
        'PRIMER_MAX_POLY_X': 100,
        'PRIMER_INTERNAL_MAX_POLY_X': 100,
        'PRIMER_SALT_MONOVALENT': 50.0,
        'PRIMER_DNA_CONC': 50.0,
        'PRIMER_MAX_NS_ACCEPTED': 0,
        'PRIMER_MAX_SELF_ANY': 12,
        'PRIMER_MAX_SELF_END': 8,
        'PRIMER_PAIR_MAX_COMPL_ANY': 12,
        'PRIMER_PAIR_MAX_COMPL_END': 8,
        'PRIMER_PRODUCT_SIZE_RANGE': [[75, 100], [100, 125], [125, 150], [150, 175], [175, 200], [200, 225]]}

In [28]:
run_p3.run_P3(global_dict=p3_test_globals,target_dict=p3_test_seq)

[{'PRIMER_LEFT': (46, 21),
  'PRIMER_LEFT_SEQUENCE': 'GCATCAGTGAGTACAGCATGC',
  'PRIMER_RIGHT': (132, 20),
  'PRIMER_RIGHT_SEQUENCE': 'TCTCCTCCTTAGCCTGCCTT',
  'TARGET_ID': 'MH1000:'},
 {'PRIMER_LEFT': (46, 21),
  'PRIMER_LEFT_SEQUENCE': 'GCATCAGTGAGTACAGCATGC',
  'PRIMER_RIGHT': (139, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CAGTGCGTCTCCTCCTTAGC',
  'TARGET_ID': 'MH1000:'},
 {'PRIMER_LEFT': (46, 21),
  'PRIMER_LEFT_SEQUENCE': 'GCATCAGTGAGTACAGCATGC',
  'PRIMER_RIGHT': (143, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CATTCAGTGCGTCTCCTCCT',
  'TARGET_ID': 'MH1000:'},
 {'PRIMER_LEFT': (46, 21),
  'PRIMER_LEFT_SEQUENCE': 'GCATCAGTGAGTACAGCATGC',
  'PRIMER_RIGHT': (130, 20),
  'PRIMER_RIGHT_SEQUENCE': 'TCCTCCTTAGCCTGCCTTTG',
  'TARGET_ID': 'MH1000:'}]

In [29]:
run_p3.run_P3(p3_test_seq, p3_test_globals)

[{'PRIMER_LEFT': (46, 21),
  'PRIMER_LEFT_SEQUENCE': 'GCATCAGTGAGTACAGCATGC',
  'PRIMER_RIGHT': (132, 20),
  'PRIMER_RIGHT_SEQUENCE': 'TCTCCTCCTTAGCCTGCCTT',
  'TARGET_ID': 'MH1000:'},
 {'PRIMER_LEFT': (46, 21),
  'PRIMER_LEFT_SEQUENCE': 'GCATCAGTGAGTACAGCATGC',
  'PRIMER_RIGHT': (139, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CAGTGCGTCTCCTCCTTAGC',
  'TARGET_ID': 'MH1000:'},
 {'PRIMER_LEFT': (46, 21),
  'PRIMER_LEFT_SEQUENCE': 'GCATCAGTGAGTACAGCATGC',
  'PRIMER_RIGHT': (143, 20),
  'PRIMER_RIGHT_SEQUENCE': 'CATTCAGTGCGTCTCCTCCT',
  'TARGET_ID': 'MH1000:'},
 {'PRIMER_LEFT': (46, 21),
  'PRIMER_LEFT_SEQUENCE': 'GCATCAGTGAGTACAGCATGC',
  'PRIMER_RIGHT': (130, 20),
  'PRIMER_RIGHT_SEQUENCE': 'TCCTCCTTAGCCTGCCTTTG',
  'TARGET_ID': 'MH1000:'}]